In [3]:


class Plot: 
    ## default contructor
    def __init__(self, index, labels, line): 
        self.unique_id  = -1
        # Somewhere labels shall be constant in PlotsManager
        self.features   = []
        
        self.features = line.strip().split(',')
        assert len(self.features) == len(labels)
        assert labels[index] == "indexField"

        self.unique_id = int(self.features[index])     

    
    def addFeatures(self,newlinelist,orderindexes,allLabels):
        self.features.extend([""] * (len(allLabels) - len(self.features)))
        for i in range(len(newlinelist)):        
           self.features[orderindexes[i]] =  newlinelist[i]
        return
          
        

        return additionalLabels
    
    ## @param[in] id : int
    def isthis(self,id):
        return self.unique_id == id

    def print(self):
        print (self.unique_id, self.features)


    def getString(self, allLabels):
       self.features.extend([""] * (len(allLabels) - len(self.features)))
       result_str = ",".join(map(str, self.features)) + "\n"
       #print(result_str)
       return result_str



In [4]:
class PlotsManager:
   ## default contructor
   def __init__(self,csvfieldplotdata):
      self.labels     = [] 
      self.indexindex = -1
      self.plots      = []
      self.unwantedlabels = ["system:index", ".geo"]

      file = open(csvfieldplotdata, 'r')
      first_line = file.readline().strip()
      if not first_line:
         raise Exception("File is empty")
      # else file is not empty
      self.labels = first_line.strip().split(',')
      try:
         self.indexindex = self.labels.index("indexField")
      except ValueError:
         print('"indexField" not found in the list of labels')

      while True:
         line = file.readline()
         if not line:  
            break
         newplot = Plot(self.indexindex,self.labels,line)
         self.plots.append(newplot)

      file.close()   

      self.startcount =0
      
      
       
       
   ## method that takes as input a unique id and text if plot already exist
   def getPlotIndex(self,id):
      idi =int(id)
      lastcount = self.startcount - 1
      for i in range(self.startcount,len(self.plots)):
         self.startcount = self.startcount+1
         if self.plots[i].isthis(idi):
            return i
      if self.startcount >=len(self.plots):
         self.startcount = 0
      for i in range(lastcount):
         if self.plots[i].isthis(idi):
            self.startcount = i
            return i 
      print("Warning: Index not found")
      return -1 # "Plot not found, wrong index given"

   # adds missing labels and let you know the indices where the new items 
   # shall be added (of the same file)
   # @returns unwantedIndex the indices of the columns to be removed
   # @returns index_list the order the items will be added once the unwantedIndex are removed
   # @notes first remove unwantedIndices because otherwice indices in index_list will not match 
   # the self.labels indices
   def find_indexes(self, B):
      index_list = []
      unwantedIndex = []

      for i, element in enumerate(B):
         if element in self.labels:
            index_list.append(self.labels.index(element))
         else:
            if element not in self.unwantedlabels: 
               self.labels.append(element)
               index_list.append(len(self.labels)-1)
            else :
               unwantedIndex.append(B.index(element))
      return index_list, unwantedIndex
   
   
   ## method that add data of given CSV file to the relevant plots
   def addInfo(self,filename):
      file = open(filename, 'r')
      first_line = file.readline().strip()
      if not first_line:
         print("WARNING: File", filename, "is empty")
         file.close()
         return
      
      # else file is not empty
      labels = first_line.split(',')
      
      ilist, unwanted = self.find_indexes(labels)
      
      filtered_labels = [labels[i] for i in range(len(labels)) if i not in unwanted]

      # Ensure "indexField" exists in filtered labels
      try:
         plotindexindex = filtered_labels.index("indexField")
      except ValueError:
         print("indexField not found in given file.",
               "The files should be the one exported from PlotToSat as that column ",
               "is appended by PlotToSat and essential for merging the files")
         file.close()
         return
      
      plotindexindex = filtered_labels.index("indexField")
      
      
      
      while True:
         line = file.readline().strip()
         if not line:
            break
         # linelist contains more items than labels because .geo column contains
         # many commas (,) but it because .geo is at the end and the extra columns do not
         # interfere with the indexes (columns) to be added to plots
         linelist = line.split(',')

         filtered_linelist = [linelist[i] for i in range(len(linelist)) if i not in unwanted]
         filtered_linelist = filtered_linelist[:len(filtered_labels)]
         #filtered_linelist = [float(item) if item != "" else item for item in filtered_linelist]

         plotIndex = self.getPlotIndex(filtered_linelist[plotindexindex])
         assert(plotIndex!=-1)
         self.plots[plotIndex].addFeatures(filtered_linelist,ilist,self.labels)

        
      return

   def print(self):
      print(self.labels)
      for plot in self.plots:
         plot.print()


   def printLabels(self):
      print(self.labels)


   def exportTo(self,csvfile):
      file = open(csvfile, "w")
      labels_string = ",".join(self.labels) + "\n"
      file.write(labels_string)
      for plot in self.plots:
         file.write(plot.getString(self.labels))
      file.close()